In [ ]:
import PIL, io, ipywidgets

import keras
import keras.applications.imagenet_utils

import numpy as np
import pandas as pd

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

inputShape = (299, 299, 3) # Standard input size for an Xception network
network = keras.applications.xception

print("[INFO] loading {}...".format(network.__name__))
model = network.Xception(weights="imagenet")
preprocess = network.preprocess_input

In [ ]:
# This is the image to be classified with above network
image_file = '/tmp/workspace/Pictures/elephants.jpg'

print("[INFO] loading and pre-processing image...")

# Load the image and display it without any preprocessing
orig_image = PIL.Image.open(image_file)
buf = io.BytesIO()
orig_image.save(buf, format='png')

img = ipywidgets.Image(value=buf.getvalue())
display(img)

# Convert the image to a numpy array to feed it into the network
image_array = keras.preprocessing.image.img_to_array(orig_image.resize(inputShape[:2]))

# Transform the image from (299, 299, 3) to (1, 299, 299, 3)
# Reason: model.predict expects arrays of images
image_array = np.expand_dims(image_array, axis=0)

# Perform necessary pre-processing on the image array
image_array = preprocess(image_array)

# Classify the image
print("[INFO] classifying image {} with {}...".format(image_file, network.__name__))
preds = model.predict(image_array)

# Decode the raw prediction result into array of 
# (ImageNetID, Description, Probability) sorted by probability
P = keras.applications.imagenet_utils.decode_predictions(preds, top=6)

results=[{'rank': i+1, 'score': prob*100.0, 'class': np.where(preds == prob)[1][0], 'id': imagenetID, 'label': label}
         for (i, (imagenetID, label, prob)) in enumerate(P[0])]

p = pd.DataFrame(results).set_index('rank')
print(p)